In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.layers import Conv2D
from time import time
from datetime import datetime
from keras.callbacks import TensorBoard, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
df1 = pd.read_hdf('df_dataDOL1.hdf', key='hdf')
df2 = pd.read_hdf('df_dataDOL2.hdf', key='hdf')
df3 = pd.read_hdf('df_dataDOL3.hdf', key='hdf')

df_total = pd.concat([df1,df2,df3],  ignore_index=True)
df_total.head()

,Data,Contratos Abertos,Contratos Fechados,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,Última oferta de compra,Última oferta de venda
0,29/07/2019,851537,719017,27192,226625,42939771125,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,145520,3.7820,3.7825
1,26/07/2019,875047,851537,32128,309480,58414828000,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,125890,3.7765,3.7780
2,25/07/2019,890941,875047,41651,381060,72105906125,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,129780,3.7795,3.7805
3,24/07/2019,883412,890941,26137,269620,50725252625,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,31160,3.7735,3.7755
4,23/07/2019,880504,883412,33586,313840,59121712875,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,356000,3.7750,3.7770


In [3]:
df = df_total.copy()

df['Dif_contratos'] = df['Contratos Abertos'] -df['Contratos Fechados']
df['Dif_minmax'] = (df['Máximo'] - df['Mínimo'])*1000
df['Dif_abert_ultimo'] = (df['Último Preço'] - df['Abertura'])*1000
df['Dif_abert_ajuste'] = (df['Ajuste'] - df['Abertura'])*1000
df['Dif_abert_min'] = (df['Abertura'] - df['Mínimo'])*1000
df['Dif_abert_max'] = (df['Abertura'] - df['Máximo'])*1000
df['Dif_abert_medio'] = (df['Abertura'] - df['Médio'])*1000
df['Dif_ajuste_ultimo'] = (df['Ajuste'] - df['Último Preço'])*1000
df['Dif_ajuste_medio'] = (df['Ajuste'] - df['Médio'])*1000
df['Dif_ultimacomp_ultimavend'] = (df['Última oferta de venda'] - df['Última oferta de compra'])*1000

df['Contratos Negociados'] = df['Contratos Negociados']/10000
df['Número Negócios'] = df['Número Negócios']/100
df['Volume'] = df['Volume']/1000000000


df = df.drop(columns = ['Contratos Abertos', 'Contratos Fechados'])


df_feature = df.set_index('Data') #Usado depois do treinamento


df.head()

,Data,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,...,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend
0,29/07/2019,271.92,22.6625,42.939771,3.7850,3.7775,3.8015,3.789499,3.7820,3.784797,...,132520,24.0,-3.0,-0.203,7.5,-16.5,-4.499,2.797,-4.702,0.5
1,26/07/2019,321.28,30.9480,58.414828,3.7800,3.7610,3.7955,3.775030,3.7760,3.770245,...,23510,34.5,-4.0,-9.755,19.0,-15.5,4.970,-5.755,-4.785,1.5
2,25/07/2019,416.51,38.1060,72.105906,3.7680,3.7570,3.8070,3.784490,3.7795,3.782834,...,15894,50.0,11.5,14.834,11.0,-39.0,-16.490,3.334,-1.656,1.0
3,24/07/2019,261.37,26.9620,50.725253,3.7675,3.7540,3.7795,3.762721,3.7735,3.769856,...,-7529,25.5,6.0,2.356,13.5,-12.0,4.779,-3.644,7.135,2.0
4,23/07/2019,335.86,31.3840,59.121713,3.7450,3.7430,3.7820,3.767634,3.7750,3.772972,...,-2908,39.0,30.0,27.972,2.0,-37.0,-22.634,-2.028,5.338,2.0


In [4]:
df_abert_old = df.copy()

#1 se o Ajuste for maior que a abertura


df_abert_old['target'] = df_abert_old.Ajuste.shift(-1) - df_abert_old['Ajuste']
df_abert_old['target'] = df_abert_old['target'].apply(lambda x: 1 if x > 0 else 0)
#df_abert_old = df_abert_old.drop(df_abert_old.tail(1).index,inplace=True)

df_abert_old['Média'] = (df_abert_old['Último Preço'] + df_abert_old['Abertura'] + df_abert_old['Máximo'] + df_abert_old['Mínimo'] + df_abert_old['Médio'] + df_abert_old['Ajuste'] + df_abert_old['Última oferta de compra']+ df_abert_old['Última oferta de venda'])/8
df_abert_old = df_abert_old.drop(columns = ['Último Preço','Abertura','Máximo','Mínimo','Médio','Última oferta de compra','Última oferta de venda','Contratos Negociados', 'Ajuste'])


df_change = df_abert_old[['target']]
df_abert_old = df_abert_old.drop(columns = 'target')
df_abert_old['target'] = df_change

df_abert_old.head()

,Data,Número Negócios,Volume,Var pontos,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,Média,target
0,29/07/2019,271.92,42.939771,145520,132520,24.0,-3.0,-0.203,7.5,-16.5,-4.499,2.797,-4.702,0.5,3.785600,0
1,26/07/2019,321.28,58.414828,125890,23510,34.5,-4.0,-9.755,19.0,-15.5,4.970,-5.755,-4.785,1.5,3.776534,1
2,25/07/2019,416.51,72.105906,129780,15894,50.0,11.5,14.834,11.0,-39.0,-16.490,3.334,-1.656,1.0,3.779853,0
3,24/07/2019,261.37,50.725253,31160,-7529,25.5,6.0,2.356,13.5,-12.0,4.779,-3.644,7.135,2.0,3.769510,1
4,23/07/2019,335.86,59.121713,356000,-2908,39.0,30.0,27.972,2.0,-37.0,-22.634,-2.028,5.338,2.0,3.767201,0


In [5]:
sc = StandardScaler()
df_abert_old.iloc[:,1:-1]  = sc.fit_transform(df_abert_old.iloc[:,1:-1])



df_moeda = pd.read_csv('moedas_features.csv')

sc = StandardScaler()
df_moeda.iloc[:,1:] = sc.fit_transform(df_moeda.iloc[:,1:])

pca = PCA(n_components=3)
dataset = pca.fit_transform(df_moeda.iloc[:,1:])
df_moeda_pca = pd.DataFrame(dataset)
df_moeda_pca['Data'] = df_moeda.iloc[:,0:1]




df_abert = pd.merge(df_moeda_pca, df_abert_old, on = 'Data')



df_abert.head()

,0,1,2,Data,Número Negócios,Volume,Var pontos,Dif_contratos,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,Média,target
0,1.498562,2.807394,0.723413,14/06/2019,2.866027,2.605075,1.099924,-0.181265,1.312951,1.557156,-0.120061,-0.332606,-2.054526,-1.942578,-0.196842,-0.178956,-0.012159,2.178212,0
1,2.236543,1.093142,0.813228,13/06/2019,2.062243,2.314778,-0.649583,-0.483705,-0.004695,-0.058399,-0.185437,0.076400,0.075670,0.280736,-0.182731,-0.177075,-0.011627,2.136875,0
2,1.688401,2.248905,0.239471,12/06/2019,2.508010,2.488214,-0.637310,-0.152949,0.440706,0.335639,-0.207215,0.785345,0.125209,0.694360,-0.223930,-0.186347,-0.012159,2.146036,0
3,2.172415,1.142010,0.374627,11/06/2019,1.687984,1.587085,0.608670,-0.243370,0.459265,-0.964686,-0.258551,1.303420,0.571064,1.353530,-0.211288,-0.217766,-0.011627,2.150590,1
4,2.012424,1.412341,1.183180,10/06/2019,1.377429,1.242700,-0.457538,-0.009080,0.069538,-0.334226,-0.229939,0.676277,0.521525,0.804702,-0.213692,-0.205747,-0.010033,2.179808,0


In [6]:
df_abert = df_abert.set_index('Data')
index = df_abert.index

In [7]:
y = df_abert[['target']]
X = df_abert.drop(columns = ['target'])
X_columns = df_abert.drop(columns = ['target'])

sc = StandardScaler()
X = sc.fit_transform(X)
#X = X.values
X

array([[ 0.73532847,  1.39379166,  0.41961075, ..., -0.19596445,
        -0.01402039,  2.00404183],
       [ 1.09805362,  0.5417644 ,  0.47191894, ..., -0.19421976,
        -0.01355552,  1.96200735],
       [ 0.82863613,  1.11620821,  0.13776338, ..., -0.20281729,
        -0.01402039,  1.97132267],
       ...,
       [ 2.64513829,  0.93990605,  0.91767337, ..., -0.22194049,
        -0.01448526, -0.5714771 ],
       [ 2.91088091,  0.57818228, -1.47667747, ..., -0.21720465,
        -0.01448526, -0.55704584],
       [ 2.2662969 ,  1.88804074,  0.41997162, ..., -0.20357272,
        -0.01448526, -0.55159947]])

In [21]:
df_final = pd.DataFrame(X,index = index)

---

Verificação da previsão com quantos pontos teria ganhado de fato

1) Usando apenas a flag

2) Usando as maiores probabilidades

In [26]:
x_train, x_test, y_train, y_test = train_test_split(df_final, y, test_size=0.2)

In [27]:
r = RandomForestClassifier()
r.fit(x_train, y_train)  

predictions = r.predict(x_test)

accuracy_score(y_test, predictions)

c:\users\choice\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\choice\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8180722891566266

In [56]:
index_target = y_test.index
df_target = pd.DataFrame(predictions, index = index_target)
df_target.columns = ['target']

In [57]:
df_inicial_target = pd.merge(df_feature,df_target, left_index = True, right_index = True)

In [58]:
#Se o target é 1 significa que o ajuste do dia seguinte será menor que o do dia anterior

df = df_inicial_target.copy()
df.head()

,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,...,Dif_minmax,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,target
Data,,,,,,,,,,,,,,,,,,,,,
2002-04-09,32.66,5.1600,5.952214,2.3146,2.301,2.320,2.307060,2.302,2.303376,149510,...,19.0,-12.6,-11.224,13.6,-5.4,7.540,1.376,-3.684,0.0,1
2002-04-12,30.78,5.1226,5.928858,2.3030,2.303,2.325,2.314787,2.313,2.312181,97400,...,22.0,10.0,9.181,0.0,-22.0,-11.787,-0.819,-2.606,0.0,1
2002-04-22,31.37,4.9836,5.843040,2.3470,2.331,2.355,2.344872,2.353,2.351821,85770,...,24.0,6.0,4.821,16.0,-8.0,2.128,-1.179,6.949,0.0,1
2002-04-26,32.43,6.6932,7.910342,2.3660,2.353,2.380,2.363636,2.376,2.376931,82170,...,27.0,10.0,10.931,13.0,-14.0,2.364,0.931,13.295,0.0,1
2002-04-29,38.59,12.2618,14.540721,2.3790,2.363,2.381,2.371715,2.368,2.368794,81370,...,18.0,-11.0,-10.206,16.0,-2.0,7.285,0.794,-2.921,0.0,1


In [37]:
#df.loc[(df.target == 1) & (df.Abertura.shift(-1) < df.Ajuste), 'teste'] = df.Ajuste.shift(-1) - df.Abertura.shift(-1)
df.loc[(df.target == 0) & (df.Abertura.shift(-1) > df.Ajuste), 'teste'] = df.Abertura.shift(-1) - df.Ajuste.shift(-1)

In [42]:
df.loc[(df.target == 0) & (df.Abertura.shift(-1) > df.Ajuste)]

,Número Negócios,Contratos Negociados,Volume,Abertura,Mínimo,Máximo,Médio,Último Preço,Ajuste,Var pontos,...,Dif_abert_ultimo,Dif_abert_ajuste,Dif_abert_min,Dif_abert_max,Dif_abert_medio,Dif_ajuste_ultimo,Dif_ajuste_medio,Dif_ultimacomp_ultimavend,target,teste
Data,,,,,,,,,,,,,,,,,,,,,
30/04/2019,32.78,2.5630,5.048683,3.9335,3.9255,3.9505,3.939666,3.9455,3.945300,70990,...,12.0,11.800,8.0,-17.0,-6.166,-0.200,5.634,0.5,0,0.025528
04/02/2019,284.60,25.8030,47.449748,3.6720,3.6655,3.6935,3.677847,3.6740,3.671744,102720,...,2.0,-0.256,6.5,-21.5,-5.847,-2.256,-6.103,1.0,0,0.037505
18/10/2018,411.53,35.3495,65.490006,3.6965,3.6765,3.7350,3.705286,3.7250,3.712488,296730,...,28.5,15.988,20.0,-38.5,-8.786,-12.512,7.202,0.5,0,0.043104
08/08/2018,338.29,30.8270,58.041386,3.7700,3.7450,3.7830,3.765620,3.7815,3.775055,39250,...,11.5,5.055,25.0,-13.0,4.380,-6.445,9.435,0.5,0,-0.010851
23/07/2018,243.12,23.5945,44.751960,3.7795,3.7760,3.8065,3.793423,3.7860,3.790351,132460,...,6.5,10.851,3.5,-27.0,-13.923,4.351,-3.072,0.5,0,0.020146
16/07/2018,265.71,24.1225,46.625377,3.8470,3.8420,3.8795,3.865716,3.8665,3.870227,9430,...,19.5,23.227,5.0,-32.5,-18.716,3.727,4.511,1.5,0,0.077448
11/06/2018,429.52,44.2560,82.123720,3.7235,3.6810,3.7380,3.711303,3.7185,3.732590,196610,...,-5.0,9.090,42.5,-14.5,12.197,14.090,21.287,1.0,0,-0.010646
23/05/2018,507.11,45.4890,82.916864,3.6750,3.6195,3.6810,3.645579,3.6295,3.626017,83030,...,-45.5,-48.983,55.5,-6.0,29.421,-3.483,-19.562,0.0,0,-0.054378
13/03/2018,391.50,40.2385,65.521058,3.2685,3.2425,3.2700,3.256635,3.2640,3.262842,10310,...,-4.5,-5.658,26.0,-1.5,11.865,-1.158,6.207,0.0,0,0.010441
